# L1: NLP tasks with a simple interface 🗞️

Load your HF API key and relevant Python libraries.

In [17]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

In [18]:
# Helper function
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None,ENDPOINT_URL=os.environ['HF_API_SUMMARY_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

### How about running it locally?
The code would look very similar if you were running it locally instead of from an API. The same is true for all the models in the rest of the course, make sure to check the [Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines) documentation page

```py
from transformers import pipeline

get_completion = pipeline("summarization", model="shleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
```

## Building a text summarization app

In [19]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

get_completion(text)

[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. It is the second tallest free-standing structure in France after the Millau Viaduct.'}]

### Getting started with Gradio `gr.Interface` 

In [13]:
import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch(share=True, server_port=int(os.environ['PORT1']))

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://0c96b4554e1a585883.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


`demo.launch(share=True)` lets you create a public link to share with your team or friends.

In [4]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch(share=True, server_port=int(os.environ['PORT2']))

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://7504e5522edd9326f9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Building a Named Entity Recognition app

We are using this [Inference Endpoint](https://huggingface.co/inference-endpoints) for `dslim/bert-base-NER`, a 108M parameter fine-tuned BART model on the NER task.

### How about running it locally?

```py
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}
    
```

In [20]:
API_URL = os.environ['HF_API_NER_BASE'] #NER endpoint
text = "My name is Andrew, I'm building DeepLearningAI and I live in California"
get_completion(text, parameters=None, ENDPOINT_URL= API_URL)

[{'entity_group': 'PER',
  'score': 0.9990625,
  'word': 'Andrew',
  'start': 11,
  'end': 17},
 {'entity_group': 'ORG',
  'score': 0.89605016,
  'word': 'DeepLearningAI',
  'start': 32,
  'end': 46},
 {'entity_group': 'LOC',
  'score': 0.99969244,
  'word': 'California',
  'start': 61,
  'end': 71}]

#### gr.interface()
- Notice below that we pass in a list `[]` to `inputs` and to `outputs` because the function `fn` (in this case, `ner()`, can take in more than one input and return more than one output.
- The number of objects passed to `inputs` list should match the number of parameters that the `fn` function takes in, and the number of objects passed to the `outputs` list should match the number of objects returned by the `fn` function.

In [15]:
def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
demo.launch(share=True, server_port=int(os.environ['PORT3']))

/Users/ytchen/Documents/experimental/GenAIGradio/.venv/lib/python3.11/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://c7cd01cd09c1bbc118.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Adding a helper function to merge tokens

In [24]:
"""
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

demo.launch(share=True, server_port=int(os.environ['PORT4']))
"""

'\ndef merge_tokens(tokens):\n    merged_tokens = []\n    for token in tokens:\n        if merged_tokens and token[\'entity\'].startswith(\'I-\') and merged_tokens[-1][\'entity\'].endswith(token[\'entity\'][2:]):\n            # If current token continues the entity of the last one, merge them\n            last_token = merged_tokens[-1]\n            last_token[\'word\'] += token[\'word\'].replace(\'##\', \'\')\n            last_token[\'end\'] = token[\'end\']\n            last_token[\'score\'] = (last_token[\'score\'] + token[\'score\']) / 2\n        else:\n            # Otherwise, add the token to the list\n            merged_tokens.append(token)\n\n    return merged_tokens\n\ndef ner(input):\n    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)\n    merged_tokens = merge_tokens(output)\n    return {"text": input, "entities": merged_tokens}\n\ngr.close_all()\ndemo = gr.Interface(fn=ner,\n                    inputs=[gr.Textbox(label="Text to find entities", lines=2)

In [22]:
import os, gradio as gr
from typing import List, Dict, Any

# --- helpers ---
def _normalize_hf_output(raw: Any) -> List[Dict[str, Any]]:
    """
    Normalize HF NER outputs so each item has:
    { 'entity': str, 'word': str, 'start': int, 'end': int, 'score': float }
    Works for:
    - token-level (has 'entity' like 'B-PER', 'I-ORG')
    - aggregated (has 'entity_group' like 'PER', 'ORG')
    - nested list outputs ([[...]])
    """
    if isinstance(raw, list) and raw and isinstance(raw[0], list):
        # flatten nested lists
        flat = []
        for sub in raw:
            flat.extend(sub)
        raw = flat

    norm = []
    for t in (raw or []):
        label = t.get("entity") or t.get("entity_group") or t.get("label") or t.get("tag") or ""
        word  = t.get("word")  or t.get("token")        or t.get("text")  or ""
        start = t.get("start")
        end   = t.get("end")
        score = t.get("score", 0.0)
        # Some endpoints may not send start/end for aggregated tokens; skip if missing
        if start is None or end is None:
            continue
        norm.append({
            "entity": label,
            "word": word,
            "start": int(start),
            "end": int(end),
            "score": float(score)
        })
    return norm

def _base_label(label: str) -> str:
    return label[2:] if label.startswith(("B-", "I-")) else label

def merge_tokens(tokens: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Merge continuation tokens:
    - If B-/I- tags exist, merge consecutive I- of same base label.
    - If only grouped labels (PER/ORG/LOC, no B-/I-), we keep as-is (already merged).
    """
    merged = []
    for tok in tokens:
        ent = tok.get("entity", "")
        cur_base = _base_label(ent)

        if merged:
            prev = merged[-1]
            prev_ent = prev.get("entity", "")
            prev_base = _base_label(prev_ent)

            # Case A: true B-/I- tagging — merge I- of the same base entity when contiguous
            if ent.startswith("I-") and prev_ent.startswith(("B-","I-")) and cur_base == prev_base and tok["start"] <= prev["end"] + 1:
                prev["word"] += tok["word"].replace("##", "")
                prev["end"] = tok["end"]
                prev["score"] = (prev["score"] + tok["score"]) / 2
                # keep prev['entity'] as-is (B- or I-), label will be cleaned later
                continue

            # Case B: no B-/I- tags (entity_group like 'PER'): if exactly adjacent and same label, merge defensively
            if not ent.startswith(("B-","I-")) and not prev_ent.startswith(("B-","I-")) and cur_base and cur_base == prev_base and tok["start"] <= prev["end"] + 1:
                prev["word"] += tok["word"].replace("##", "")
                prev["end"] = tok["end"]
                prev["score"] = (prev["score"] + tok["score"]) / 2
                continue

        merged.append(tok)
    return merged

# --- your callable ---
# Make sure API_URL is defined somewhere earlier OR read from env here:
API_URL = os.getenv("HF_API_NER_BASE", "")

def ner(text: str):
    # If your get_completion supports parameters, ask for token-level output to guarantee 'entity' keys:
    # parameters={"aggregation_strategy": "none"}
    output = get_completion(text, parameters={"aggregation_strategy": "none"}, ENDPOINT_URL=API_URL)

    # But still normalize to be safe (handles both shapes)
    tokens = _normalize_hf_output(output)
    merged_tokens = merge_tokens(tokens)

    # Prepare entities for Gradio HighlightedText (expects "label", not "entity")
    entities = []
    for t in merged_tokens:
        label = _base_label(t["entity"]) or t["entity"] or "ENT"
        entities.append({"start": t["start"], "end": t["end"], "label": label})

    return {"text": text, "entities": entities}

# --- UI ---
gr.close_all()
demo = gr.Interface(
    fn=ner,
    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
    outputs=[gr.HighlightedText(label="Text with entities")],
    title="NER with dslim/bert-base-NER",
    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
    allow_flagging="never",
    examples=[
        "My name is Andrew, I'm building DeeplearningAI and I live in California",
        "My name is Poli, I live in Vienna and work at HuggingFace"
    ],
)
demo.launch(share=True, server_port=int(os.environ.get("PORT4", "7860")))


/Users/ytchen/Documents/experimental/GenAIGradio/.venv/lib/python3.11/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Closing server running on port: 7863
* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://d72003616c758dbc97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
gr.close_all()

Closing server running on port: 7863


## How to get your own Hugging Face API key (token)

Hugging Face "API keys" are called "User Access tokens".  

You can create your own User Access Tokens here: [Access Tokens](https://huggingface.co/settings/tokens).

#### Save your user access tokens to environment variables
To save your access token securely on your own machine:
- Create a `.env` file in the root directory of your project.
- Edit the file to contain the following:  
`HF_API_KEY="abc123"` replace that string with your user access token.
- Save the .env file.
- Install Python-dotenv, which allows you to run that first code cell at the top of this jupyter notebook:  
`pip install python-dotenv`


For more information on how to get your own access tokens, please see [User access tokens](https://huggingface.co/docs/hub/security-tokens#:~:text=To%20create%20an%20access%20token,you're%20ready%20to%20go!)